In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os
 

In [2]:
parent_path = r"E:\Projects\IMPACT OF E-WOM ON PURCHASE INTENTION OF KURTIS"

In [3]:
path = os.path.join(parent_path,'data','final_data.csv')
df = pd.read_csv(path)

In [4]:
df.head()

,Unnamed: 0,Rating,Review
0,0,Recommended,super quality love
1,1,Recommended,nice super
2,2,Not Recommended,poor quality
3,3,Not Recommended,colour fade first wash cloth shrink
4,4,Recommended,nice meterial tag dress sometimes delivery man...


In [5]:
df.drop(['Unnamed: 0'],inplace= True,axis = 1)

In [6]:
df['Recommendation'] = df.Rating.apply(lambda x: 1 if x=='Recommended' else 0)

In [7]:
df.groupby('Rating').describe()

Recommendation                                   
                         count mean  std  min  25%  50%  75%  max
Rating                                                           
Not Recommended         1252.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
Recommended             4700.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

## OverSampling the data (Balencing the data)

In [8]:
import imblearn
print(imblearn.__version__)

0.9.0


In [9]:
dataset = df.copy()
X = np.array(dataset.Review).reshape(-1,1)
y = np.array(dataset.Recommendation).reshape(-1,1)

In [10]:
X.shape, y.shape

((5952, 1), (5952, 1))

In [11]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy=0.5)

In [12]:
X_over,y_over = oversample.fit_resample(X,y)

In [25]:
data = pd.DataFrame({'X': X_over.flatten(),'y':y_over})

In [27]:
data.to_csv("data.csv")

In [13]:
X_over.shape, y_over.shape

((7050, 1), (7050,))

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over.reshape(-1,1), test_size=0.2, random_state=42,)

In [15]:
y_train.flatten()

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

In [16]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [17]:
# Bert layers
input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
x = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
output = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(x)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[input], outputs = [output])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [19]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [20]:
model.fit(X_train,y_train,epochs= 10)

Epoch 1/10
177/177 [==============================] - 1179s 7s/step - loss: 0.6049 - accuracy: 0.6796 - precision: 0.6854 - recall: 0.9667
Epoch 2/10
177/177 [==============================] - 1087s 6s/step - loss: 0.5435 - accuracy: 0.7278 - precision: 0.7281 - recall: 0.9493
Epoch 3/10
177/177 [==============================] - 1087s 6s/step - loss: 0.5111 - accuracy: 0.7528 - precision: 0.7541 - recall: 0.9377
Epoch 4/10
177/177 [==============================] - 1085s 6s/step - loss: 0.4882 - accuracy: 0.7699 - precision: 0.7752 - recall: 0.9258
Epoch 5/10
177/177 [==============================] - 1078s 6s/step - loss: 0.4747 - accuracy: 0.7855 - precision: 0.7904 - recall: 0.9261
Epoch 6/10
177/177 [==============================] - 1080s 6s/step - loss: 0.4617 - accuracy: 0.7933 - precision: 0.8033 - recall: 0.9166
Epoch 7/10
177/177 [==============================] - 1084s 6s/step - loss: 0.4541 - accuracy: 0.7963 - precision: 0.8054 - recall: 0.9187
Epoch 8/10
177/177 [=======